In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

In [ ]:
import requests


url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31","lon":"13.24","radiusKm":"50","limit":"16","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"count":2,"items":[{"icao":"EDDB","iata":"BER","name":"Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE","timeZone":"Europe/Berlin"},{"icao":"EDDT","iata":"TXL","name":"Berlin -Tegel","shortName":"-Tegel","municipalityName":"Berlin","location":{"lat":52.5597,"lon":13.287699},"countryCode":"DE","timeZone":"Europe/Berlin"}]}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [ ]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 2,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'},
  {'icao': 'EDDT',
   'iata': 'TXL',
   'name': 'Berlin -Tegel',
   'shortName': '-Tegel',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.5597, 'lon': 13.287699},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'}]}

We can now turn this into a dataframe using `.json_normalize()`

In [ ]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699


Let's now use this for the latitude and longitude of multiple cities

In [ ]:
def get_airports(latitudes, longitudes):
  # API headers
  headers = {
      "X-RapidAPI-Key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }

  querystring = {"withFlightInfoOnly": "true"}

  # DataFrame to store results
  all_airports = []

  for lat, lon in zip(latitudes, longitudes):
    # Construct the URL with the latitude and longitude
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"

    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      data = response.json()
      airports = pd.json_normalize(data.get('items', []))
      all_airports.append(airports)

  return pd.concat(all_airports, ignore_index=True)

In [ ]:
# coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

get_airports(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
2,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
3,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
4,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
5,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
6,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
7,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
8,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
9,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333


# **Arrivals information**


In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2025-03-24T20:00/2025-03-24T22:00"

querystring = {"withLeg":"true",
               "direction":"Arrival",
               "withCancelled":"false",
               "withCodeshared":"true",
               "withCargo":"false",
               "withPrivate":"false"}

headers = {
	"x-rapidapi-key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
flights_json = response.json()
print (flights_json)

{'arrivals': [{'departure': {'airport': {'icao': 'EVRA', 'iata': 'RIX', 'name': 'Riga', 'timeZone': 'Europe/Riga'}, 'scheduledTime': {'utc': '2025-03-24 16:55Z', 'local': '2025-03-24 18:55+02:00'}, 'revisedTime': {'utc': '2025-03-24 16:55Z', 'local': '2025-03-24 18:55+02:00'}, 'runwayTime': {'utc': '2025-03-24 17:26Z', 'local': '2025-03-24 19:26+02:00'}, 'terminal': 'C', 'gate': 'C6', 'runway': '36', 'quality': ['Basic', 'Live']}, 'arrival': {'scheduledTime': {'utc': '2025-03-24 18:45Z', 'local': '2025-03-24 19:45+01:00'}, 'revisedTime': {'utc': '2025-03-24 19:01Z', 'local': '2025-03-24 20:01+01:00'}, 'terminal': '2', 'gate': 'Z36', 'baggageBelt': 'C1', 'quality': ['Basic', 'Live']}, 'number': 'FR 2637', 'callSign': 'RYR2637', 'status': 'Arrived', 'codeshareStatus': 'IsOperator', 'isCargo': False, 'aircraft': {'reg': 'SP-RNH', 'modeS': '48C1A7', 'model': 'Boeing 737'}, 'airline': {'name': 'Ryanair', 'iata': 'FR', 'icao': 'RYR'}}, {'departure': {'airport': {'icao': 'LFSB', 'iata': 'BSL'

In [ ]:
flights_json

{'arrivals': [{'departure': {'airport': {'icao': 'EVRA',
     'iata': 'RIX',
     'name': 'Riga',
     'timeZone': 'Europe/Riga'},
    'scheduledTime': {'utc': '2025-03-24 16:55Z',
     'local': '2025-03-24 18:55+02:00'},
    'revisedTime': {'utc': '2025-03-24 16:55Z',
     'local': '2025-03-24 18:55+02:00'},
    'runwayTime': {'utc': '2025-03-24 17:26Z',
     'local': '2025-03-24 19:26+02:00'},
    'terminal': 'C',
    'gate': 'C6',
    'runway': '36',
    'quality': ['Basic', 'Live']},
   'arrival': {'scheduledTime': {'utc': '2025-03-24 18:45Z',
     'local': '2025-03-24 19:45+01:00'},
    'revisedTime': {'utc': '2025-03-24 19:01Z',
     'local': '2025-03-24 20:01+01:00'},
    'terminal': '2',
    'gate': 'Z36',
    'baggageBelt': 'C1',
    'quality': ['Basic', 'Live']},
   'number': 'FR 2637',
   'callSign': 'RYR2637',
   'status': 'Arrived',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'reg': 'SP-RNH', 'modeS': '48C1A7', 'model': 'Boeing 737'},
   'airli

In [ ]:
flights_json.keys()

dict_keys(['arrivals'])

In [ ]:
flights_json["arrivals"][0]

{'departure': {'airport': {'icao': 'EVRA',
   'iata': 'RIX',
   'name': 'Riga',
   'timeZone': 'Europe/Riga'},
  'scheduledTime': {'utc': '2025-03-24 16:55Z',
   'local': '2025-03-24 18:55+02:00'},
  'revisedTime': {'utc': '2025-03-24 16:55Z',
   'local': '2025-03-24 18:55+02:00'},
  'runwayTime': {'utc': '2025-03-24 17:26Z',
   'local': '2025-03-24 19:26+02:00'},
  'terminal': 'C',
  'gate': 'C6',
  'runway': '36',
  'quality': ['Basic', 'Live']},
 'arrival': {'scheduledTime': {'utc': '2025-03-24 18:45Z',
   'local': '2025-03-24 19:45+01:00'},
  'revisedTime': {'utc': '2025-03-24 19:01Z',
   'local': '2025-03-24 20:01+01:00'},
  'terminal': '2',
  'gate': 'Z36',
  'baggageBelt': 'C1',
  'quality': ['Basic', 'Live']},
 'number': 'FR 2637',
 'callSign': 'RYR2637',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'reg': 'SP-RNH', 'modeS': '48C1A7', 'model': 'Boeing 737'},
 'airline': {'name': 'Ryanair', 'iata': 'FR', 'icao': 'RYR'}}

In [ ]:
flights_json["arrivals"][0].keys()

dict_keys(['departure', 'arrival', 'number', 'callSign', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [ ]:
df= pd.json_normalize(flights_json["arrivals"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   number                         33 non-null     object
 1   callSign                       30 non-null     object
 2   status                         33 non-null     object
 3   codeshareStatus                33 non-null     object
 4   isCargo                        33 non-null     bool  
 5   departure.airport.icao         33 non-null     object
 6   departure.airport.iata         33 non-null     object
 7   departure.airport.name         33 non-null     object
 8   departure.airport.timeZone     33 non-null     object
 9   departure.scheduledTime.utc    33 non-null     object
 10  departure.scheduledTime.local  33 non-null     object
 11  departure.revisedTime.utc      29 non-null     object
 12  departure.revisedTime.local    29 non-null     object
 13  departu

In [ ]:
columns_to_keep = [
    "number",
    "status",
    "departure.gate",
    "airline.icao",
    "departure.gate",
    "departure.gate"

]

# Filter the DataFrame
df_filtered = df[columns_to_keep]

# Optional: Preview the cleaned DataFrame
df_filtered

,number,status,departure.gate,airline.icao,departure.gate,departure.gate
0,FR 2637,Arrived,C6,RYR,C6,C6
1,U2 1187,Arrived,84,EZY,84,84
2,DS 1187,Arrived,84,EZS,84,84
3,LG 9473,Arrived,B04,LGL,B04,B04
4,VL 1954,Arrived,NaN,LHX,NaN,NaN
5,VY 1886,Arrived,B39,VLG,B39,B39
6,FR 1147,Arrived,NaN,RYR,NaN,NaN
7,U2 8625,Arrived,NaN,EZY,NaN,NaN
8,FR 137,Arrived,NaN,RYR,NaN,NaN
9,EC 5106,Arrived,D45,EJU,D45,D45


In [ ]:
def flight_arrival_for_airports(airport_icaos):
    querystring = {
        "withLeg": "false",
        "direction": "Arrival",
        "withCancelled": "false",
        "withCodeshared": "true",
        "withCargo": "true",
        "withPrivate": "false",
        "withLocation": "false"
    }

    headers = {
        "x-rapidapi-key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }

    all_arrivals = []

    for icao in airport_icaos:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/2025-03-24T20:00/2025-03-24T22:00"
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json()
            arrivals = pd.json_normalize(data.get('arrivals', []))
            if not arrivals.empty:
                arrivals["airport_icao"] = icao  # Add source airport column
                all_arrivals.append(arrivals)
        else:
            print(f"Failed to get data for {icao}: {response.status_code}")

    # Concatenate all results into one DataFrame
    if all_arrivals:
        return pd.concat(all_arrivals, ignore_index=True)
    else:
        return pd.DataFrame()

In [ ]:
airport_icaos = ['EDDB', 'EGKK']

flight_arrival_for_airports(airport_icaos)

,number,callSign,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.airport.timeZone,movement.scheduledTime.utc,...,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,airline.iata,airline.icao,movement.runwayTime.utc,movement.runwayTime.local,movement.runway,airport_icao
0,FR 2637,RYR2637,Arrived,IsOperator,False,EVRA,RIX,Riga,Europe/Riga,2025-03-24 18:45Z,...,SP-RNH,48C1A7,Boeing 737,Ryanair,FR,RYR,NaN,NaN,NaN,EDDB
1,DS 1187,EZS23BQ,Arrived,IsCodeshared,False,LFSB,BSL,Bâle/Mulhouse,Europe/Paris,2025-03-24 19:30Z,...,OE-ICF,440019,Airbus A320,Easyjet Switzerland,DS,EZS,2025-03-24 19:03Z,2025-03-24 20:03+01:00,07R,EDDB
2,U2 1187,EZS23BQ,Arrived,IsOperator,False,LFSB,BSL,Bâle/Mulhouse,Europe/Paris,2025-03-24 19:30Z,...,OE-ICF,440019,Airbus A320,easyJet,U2,EZY,2025-03-24 19:03Z,2025-03-24 20:03+01:00,07R,EDDB
3,LG 9473,LGL9473,Arrived,IsOperator,False,ELLX,LUX,Luxembourg,Europe/Luxembourg,2025-03-24 19:20Z,...,LX-LQC,4D00D7,De Havilland Canada DHC-8-400 Dash 8Q,Luxair,LG,LGL,2025-03-24 19:07Z,2025-03-24 20:07+01:00,07L,EDDB
4,VL 1954,LHX1954,Arrived,IsOperator,False,EDDM,MUC,Munich,Europe/Berlin,2025-03-24 19:15Z,...,D-AIJN,3C654E,Airbus A320 NEO,VL,VL,LHX,NaN,NaN,07R,EDDB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,BY 771,TOM771,Arrived,IsOperator,False,HEGN,HRG,Hurghada,Africa/Cairo,2025-03-24 21:50Z,...,G-TUMX,407E7B,Boeing 737 MAX 8,TUI,BY,TOM,NaN,NaN,NaN,EGKK
70,U2 8764,EZY78CZ,Arrived,IsOperator,False,LMML,MLA,Luqa,Europe/Malta,2025-03-24 21:50Z,...,G-UZMK,40812D,Airbus A321 NEO,easyJet,U2,EZY,NaN,NaN,NaN,EGKK
71,U2 8314,EZY98NA,Arrived,IsOperator,False,LIMC,MXP,Milan,Europe/Rome,2025-03-24 21:50Z,...,G-EZDV,405D0F,Airbus A319,easyJet,U2,EZY,NaN,NaN,26L,EGKK
72,U2 8480,EZY73UW,Arrived,IsOperator,False,LFSB,BSL,Bâle/Mulhouse,Europe/Paris,2025-03-24 21:55Z,...,G-EJCA,407E5F,Airbus A320,easyJet,U2,EZY,NaN,NaN,NaN,EGKK
